<a href="https://colab.research.google.com/github/mengyaoo/FNL_GenesSelection/blob/main/05_TC1_LASSO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load the libraries

In [1]:
from __future__ import print_function

import os, sys, gzip, glob, json, time, argparse
import pandas as pd
import numpy as np
import keras
from pandas.io.json import json_normalize

from pandas.io.json import json_normalize
from keras.utils import to_categorical
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

from keras.utils import to_categorical
from keras import backend as K
from keras.layers import Input, Dense, Dropout, Activation, Conv1D, MaxPooling1D, Flatten
from keras import optimizers
from keras.optimizers import SGD, Adam, RMSprop
from keras.models import Sequential, Model, model_from_json, model_from_yaml
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau

from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import sklearn.manifold as sk_manif
from keras.utils import to_categorical
from google.colab import drive

import matplotlib.pyplot as plt
%matplotlib inline

# Data Preparation   

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Read features and output files 
TC1data15 = pd.read_csv("/content/drive/My Drive/FNL_TC1/TC1-S1-data15-genename.tsv", sep="\t", low_memory = False)
#TC1data15 = sfeatures1


In [4]:
outcome = pd.read_csv('/content/drive/My Drive/FNL_TC1/TC1-outcome-data15-projectname.tsv', sep='\t')
outcome

,Project_id_name,Project_id
0,TCGA-OV,10
1,TCGA-OV,10
2,TCGA-OV,10
3,TCGA-OV,10
4,TCGA-OV,10
...,...,...
4495,TCGA-LIHC,7
4496,TCGA-LIHC,7
4497,TCGA-LIHC,7
4498,TCGA-LIHC,7


In [5]:
def encode(data): 
    print('Shape of data (BEFORE encode): %s' % str(data.shape))
    encoded = to_categorical(data)
    print('Shape of data (AFTER  encode): %s\n' % str(encoded.shape))
    return encoded

In [ ]:
# outcome = encode(outcome['Project_id'])

# Logistic + LASSO

In [ ]:
lasso = LogisticRegression(C=1,penalty='l1', solver='saga', multi_class='multinomial')
lasso.fit(TC1data15, outcome['Project_id'])


In [ ]:
importance = lasso.coef_[0]
# summarize feature importance
for i,v in enumerate(importance):
	print('Feature: %0d, Score: %.5f' % (i,v))
# plot feature importance
plt.bar([x for x in range(len(importance))], importance)
plt.show()

In [ ]:
scaler = StandardScaler()
scaler.fit(TC1data15)

model = SelectFromModel(LogisticRegression(C=1, penalty='l1', solver='saga',multi_class='multinomial'))
model.fit(TC1data15.fillna(0), outcome['Project_id'])

In [24]:
selected_feat = TC1data15.columns[(model.get_support())]
print('total features: {}'.format((TC1data15.shape[1])))
print('selected features: {}'.format(len(selected_feat)))
print('features with coefficients shrank to zero: {}'.format(
      np.sum(model.estimator_.coef_ == 0)))

total features: 60483
selected features: 20647
features with coefficients shrank to zero: 838971


In [22]:
TC1data15_selected = model.transform(TC1data15.fillna(0))
TC1data15_selected.shape

(4500, 20647)

# ConvNN(LASSO) - top 300

## Train/Test split    

In [ ]:
# from keras.utils import to_categorical
# outcome = encode(outcome['Project_id'])

In [ ]:
TC1data15_selected = TC1data15.loc[:,sum_loading.sort_values(ascending=False)[:300].index]

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(TC1data15_selected, 
                                                    outcome, 
                                                    train_size=0.75, 
                                                    test_size=0.25, 
                                                    random_state=123, 
                                                    stratify = outcome)

In [ ]:
X_train

,OLFM4,IGHD,CST1,MINOS1P3,PRSS21,COX20P1,H19,MTND4P12,NTS,LTF,PLA2G2A,SST,RP11-109M17.2,ERAP2,HCG4P5,UPK1B,MSMB,COX6CP1,DEFB1,FABP4,NLRP2,IGLV8-61,CXCL5,RPS28P7,MSLN,EEF1A2,RP11-467L13.5,IGLV10-54,IGLV4-60,C1DP1,PIGR,RP11-305M3.2,CD177,IGHG4,FDCSP,LGALS7,SPESP1,IGF2,SCGB1A1,CXCL14,...,CTAG2,RP11-206M11.7,GPX2,FCGBP,CCL18,IGKV6D-21,PIP,KLK11,KLK6,HOXC10,IGHV4-4,FGB,NEFL,GAL,IGLV3-21,VNN1,SLPI,AMTN,UGT2B17,PRSS3,SAA1,PCDHGA10,IGHV3-43,RP11-297P16.4,CA12,C2orf54,WDR72,IGKV5-2,NMU,NUPR1L,RPL39L,CXCL6,MGST1,MMP3,CRABP2,MMP12,IGHA2,GPAT2,CTA-384D8.31,VTCN1
849,0.000000,3.451720,1.924427,1.214701,0.034128,0.796983,0.000000,0.241386,0.000000,0.955123,1.115573,0.000000,0.000000,1.494122,0.000000,0.000000,0.000000,0.358672,0.534561,0.000000,0.000000,2.116817,0.425798,2.638663,3.431951,1.762916,0.000000,2.168452,1.768109,1.096429,1.691947,0.938032,0.000000,2.661443,1.421802,0.000000,0.000000,1.008352,1.507239,1.348174,...,0.0,0.000000,2.520775,0.925476,2.489023,1.107416,0.000000,0.759562,0.000000,1.406353,0.531705,0.959357,0.168666,0.000000,3.052650,0.231438,3.612812,0.000000,0.000000,0.000000,0.704465,0.000000,1.472290,0.000000,0.354215,1.127222,0.082539,1.212567,0.687382,0.000000,1.187233,0.000000,2.715825,0.000000,2.681216,0.724169,3.432900,0.000000,0.143028,0.685608
1050,0.000000,0.891166,0.070649,0.000000,0.000000,1.087995,0.589354,2.247298,0.000000,2.601139,0.000000,0.000000,0.000000,1.327797,0.895526,0.411449,0.434593,1.299270,0.561074,0.000000,0.000000,0.447410,2.245163,2.926227,1.924036,1.196526,1.149015,0.201088,1.191099,0.937306,0.016960,0.140908,0.000000,0.820715,0.359019,0.000000,0.000000,0.658734,0.000000,1.471142,...,0.0,2.331532,0.677756,1.400181,2.060181,0.417925,2.267888,0.000000,1.414726,0.277530,0.000000,0.000000,0.000000,2.163065,1.526897,1.169651,2.694524,0.000000,0.000000,1.531403,1.927743,0.000000,0.354370,0.000000,1.289541,0.000000,0.000000,0.339807,0.332531,0.000000,2.164836,0.666409,1.967046,0.658254,2.199999,2.075464,2.382251,0.000000,2.269779,2.263385
835,0.000000,1.241689,2.417806,1.595289,1.401257,0.267332,1.108554,2.232516,3.162791,0.936211,0.000000,0.000000,0.924735,0.631624,0.883749,0.543921,0.031125,0.186532,1.641631,0.629357,0.000000,1.326851,0.000000,3.019834,0.130461,0.770522,0.000000,0.720105,0.000000,0.000000,1.153434,0.253478,0.192738,2.192938,1.541400,1.804994,1.596873,1.000693,2.067914,3.207606,...,0.0,0.000000,3.450378,0.261245,1.577894,0.000000,0.000000,0.996138,0.781892,0.982658,0.561592,0.000000,0.606858,0.000000,1.821441,0.000000,2.502938,2.232032,0.115347,0.443945,0.334279,0.018825,0.328638,1.011512,1.842113,0.000000,1.208723,0.000000,0.775478,0.000000,1.575012,0.000000,0.678401,0.088454,2.865003,1.300814,2.024221,0.000000,0.000000,1.764685
4447,0.000000,0.314609,2.389053,0.000000,0.000000,0.000000,1.863976,0.590006,0.279646,0.000000,0.083925,0.000000,0.107764,1.177361,0.000000,0.000000,0.000000,0.000000,0.000000,0.283025,0.000000,2.481546,0.580356,3.275798,0.312685,2.387843,0.735262,0.063985,1.574975,0.000000,0.000000,0.000000,0.103225,2.612143,0.000000,0.000000,0.000000,2.591494,0.000000,2.622154,...,0.0,0.000000,2.273950,0.375970,1.374103,0.315585,0.000000,0.000000,0.000000,0.000000,0.861459,0.000000,0.000000,1.171934,2.621442,0.000000,0.674940,0.000000,1.265490,0.713333,1.032651,0.000000,1.493938,0.000000,1.660267,0.378461,0.000000,1.761213,0.000000,0.000000,0.366889,1.295114,1.782123,1.179261,2.802413,1.313344,3.138005,0.000000,0.000000,0.516333
53,0.000000,0.124001,0.848674,0.000000,2.321561,0.514690,0.000000,1.153314,0.320263,0.424957,0.372927,1.334508,0.536945,0.704182,1.708493,0.261167,0.000000,0.166718,2.422981,0.000000,0.014891,0.890756,0.685729,2.145886,2.660830,1.522546,1.229384,0.970287,0.054391,0.125379,1.278695,0.166718,0.000000,0.708005,0.000000,0.000000,0.000000,2.255922,0.000000,0.676345,...,0.0,0.000000,0.000000,0.466422,0.823782,0.000000,0.538180,1.701283,2.459654,0.842577,0.326178,0.000000,0.000000,0.737395,1.455956,1.078100,3.761177,0.000000,0.000000,0.000000,2.74508

## CONV1D 

In [ ]:
# parameters  
activation='relu'
batch_size=20
# Number of sites
classes=15
drop = 0.1
feature_subsample = 0
loss='categorical_crossentropy'
# metrics='accuracy'
out_act='softmax'
pool=[1, 10]
# optimizer='sgd'
shuffle = False 
epochs=100

optimizer = optimizers.SGD(lr=0.1)
metrics = ['acc']

In [ ]:
# X_train shape: (3375, 300)
# X_test shape:  (1125, 300)
# Y_train shape: (3375,15)
# Y_test shape:  (1125,15)

# 300
x_train_len = X_train.shape[1]   

X_train = np.expand_dims(X_train, axis=2)
X_test = np.expand_dims(X_test, axis=2)

# X_train shape: (3375, 300, 1)
# X_test shape:  (1125, 300, 1)

filters = 128 
filter_len = 20 
stride = 1 

# inside pool_list loop
pool_list = [1,10]

K.clear_session()

In [ ]:
model = Sequential()

# model.add  CONV1D
model.add(Conv1D(filters = filters, 
                 kernel_size = filter_len, 
                 strides = stride, 
                 padding='valid', 
                 input_shape=(x_train_len, 1)))

# x_train_len = 300
# Activation
model.add(Activation('relu'))

# MaxPooling
model.add(MaxPooling1D(pool_size = 1))

filters = 128
filter_len = 10 
stride = 1 
# Conv1D
model.add(Conv1D(filters=filters, 
                 kernel_size=filter_len, 
                 strides=stride, 
                 padding='valid'))
# Activation
model.add(Activation('relu'))

# MaxPooling
model.add(MaxPooling1D(pool_size = 10))
model.add(Flatten())

model.add(Dense(200))

# activation 
# model.add(Activation('relu')) # SR
model.add(Activation(activation))
#dropout
model.add(Dropout(0.1))

model.add(Dense(20))
# activation
# model.add(Activation('relu')) # SR
model.add(Activation(activation))

#dropout
model.add(Dropout(0.1))

model.add(Dense(15))
model.add(Activation(out_act))

model.compile( loss= loss, 
              optimizer = optimizer, 
              metrics = metrics )
model.summary()



Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 281, 128)          2688      
_________________________________________________________________
activation (Activation)      (None, 281, 128)          0         
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 281, 128)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 272, 128)          163968    
_________________________________________________________________
activation_1 (Activation)    (None, 272, 128)          0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 27, 128)           0         
_________________________________________________________________
flatten (Flatten)            (None, 3456)              0

In [ ]:
# save
save = '.'
output_dir = "/content/drive/My Drive/FNL_TC1/Model"

if not os.path.exists(output_dir):
        os.makedirs(output_dir)

model_name = 'tc1_lasso300'
path = '{}/{}.autosave.model.h5'.format(output_dir, model_name)
checkpointer = ModelCheckpoint(filepath=path,
                               verbose=1,
                               save_weights_only=True,
                               save_best_only=True)

csv_logger = CSVLogger('{}/training.log'.format(output_dir))

In [ ]:
# SR: change epsilon to min_delta
reduce_lr = ReduceLROnPlateau(monitor='val_loss', 
                              factor=0.1, 
                              patience=10, 
                              verbose=1, mode='auto', 
                              min_delta=0.0001, 
                              cooldown=0, 
                              min_lr=0)

In [ ]:
# batch_size = 20 
history = model.fit(X_train, Y_train, batch_size=batch_size, 
                    epochs=epochs, verbose=1, validation_data=(X_test, Y_test), 
                    callbacks = [checkpointer, csv_logger, reduce_lr])

score = model.evaluate(X_test, Y_test, verbose=0)

Epoch 1/100
169/169 [==============================] - ETA: 0s - loss: 2.0384 - acc: 0.3369
Epoch 00001: val_loss improved from inf to 0.64536, saving model to Model/tc1_pca300.autosave.model.h5
169/169 [==============================] - 13s 75ms/step - loss: 2.0384 - acc: 0.3369 - val_loss: 0.6454 - val_acc: 0.8027
Epoch 2/100
169/169 [==============================] - ETA: 0s - loss: 0.5012 - acc: 0.8483
Epoch 00002: val_loss improved from 0.64536 to 0.36220, saving model to Model/tc1_pca300.autosave.model.h5
169/169 [==============================] - 12s 73ms/step - loss: 0.5012 - acc: 0.8483 - val_loss: 0.3622 - val_acc: 0.8969
Epoch 3/100
169/169 [==============================] - ETA: 0s - loss: 0.2710 - acc: 0.9230
Epoch 00003: val_loss improved from 0.36220 to 0.28531, saving model to Model/tc1_pca300.autosave.model.h5
169/169 [==============================] - 12s 71ms/step - loss: 0.2710 - acc: 0.9230 - val_loss: 0.2853 - val_acc: 0.9138
Epoch 4/100
169/169 [=================

In [ ]:
print('Test score:', score[0])
print('Test accuracy:', score[1])

In [ ]:
plt.plot(history.history['acc'],label="accuracy")
plt.plot(history.history['val_acc'],label="val_accuracy")
plt.plot(history.history['loss'],label="loss")
plt.plot(history.history['val_loss'],label="val_loss")
plt.legend()
plt.title("Top 300 features")
plt.ylabel('epoch')


In [ ]:
# JSON JSON
# serialize model to json
json_model = model.to_json()

# save the model architecture to JSON file
with open('{}/{}.model.json'.format(output_dir, model_name), 'w') as json_file:
    json_file.write(json_model)


# YAML YAML
# serialize model to YAML
model_yaml = model.to_yaml()

# save the model architecture to YAML file
with open("{}/{}.model.yaml".format(output_dir, model_name), "w") as yaml_file:
    yaml_file.write(model_yaml)


# WEIGHTS HDF5
# serialize weights to HDF5
model.save_weights("{}/{}.model.h5".format(output_dir,model_name))
print("Saved model to disk")

In [ ]:
# load weights into new model
loaded_model_yaml.load_weights('{}/{}.model.h5'.format(output_dir,model_name))
print("Loaded yaml model from disk")

# evaluate loaded model on test data
loaded_model_yaml.compile(loss=loss,optimizer=optimizer, metrics=metrics) 
score_yaml = loaded_model_yaml.evaluate(X_test, Y_test, verbose=0)

print('yaml Test score:', score_yaml[0])
print('yaml Test accuracy:', score_yaml[1])

print("yaml %s: %.2f%%" % (loaded_model_yaml.metrics_names[1], score_yaml[1]*100))